In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from lib import model_device_io as io
from lib import data_loader as dl

In [2]:
#encode non float scripts with one-hot vec and show encoding dictionary
tensors = prep.GetFeatureVectorsAndLabels(fnTrain = 'data/train.csv', fnTest = 'data/test.csv')

Encoding completed

Using the Following Notation: 
 {'Check-Out, 0': 0, 'Check-Out, 1': 1, 'Canceled, 0': 2, 'Canceled, 1': 3, 'No-Show, 0': 4, 'No-Show, 1': 5}


In [3]:
#save tensors
fnList = ['train_data_tensor.pt', 'test_data_tensor.pt', 'train_label_tensor_adr.pt', 'train_label_tensor_rev.pt']
for tensor, fn in zip(tensors, fnList):
    torch.save(tensor, os.path.join('data', fn))
    
#reload to check if saving succeeded
for fn in fnList:
    fn = os.path.join('data', fn)
    print(fn, '\t', torch.load(fn).shape)

data/train_data_tensor.pt 	 torch.Size([91531, 258])
data/test_data_tensor.pt 	 torch.Size([27859, 258])
data/train_label_tensor_adr.pt 	 torch.Size([91531, 1])
data/train_label_tensor_rev.pt 	 torch.Size([91531, 1])


In [4]:
#split test set from train set for validations
train_data_tensor, _,  train_label_tensor_adr, train_label_tensor_rev = tensors
tensors = prep.RandomSplit(train_data_tensor, train_label_tensor_adr, train_label_tensor_rev, 10000)

In [5]:
#save tensors
fnList = ['test/data.pt', 'train/data.pt', 'test/adr.pt', 'train/adr.pt', 'test/rev.pt', 'train/rev.pt']
for tensor, fn in zip(tensors, fnList):
    torch.save(tensor, os.path.join('data', fn))

#reload to check if saving succeeded   
for fn in fnList:
    fn = os.path.join('data', fn)
    print(fn, '\t', torch.load(fn).shape)

data/test/data.pt 	 torch.Size([10000, 258])
data/train/data.pt 	 torch.Size([81531, 258])
data/test/adr.pt 	 torch.Size([10000, 1])
data/train/adr.pt 	 torch.Size([81531, 1])
data/test/rev.pt 	 torch.Size([10000, 1])
data/train/rev.pt 	 torch.Size([81531, 1])
